In [2]:
import pandas as pd
import matplotlib as mpl
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.probability import FreqDist
import time
import jieba
import codecs
import gc
import tqdm
import gensim
from gensim import corpora, models, similarities
import pyLDAvis.gensim
from collections import defaultdict
import string

In [3]:
import warnings
warnings.filterwarnings("ignore")

# Chatbot pipeline

In [4]:
#generate class for segmentation
class Seg(object):
#     stopword_filepath = "stopword.txt"

    def __init__(self):
        self.stopwords = set()
#         self.read_in_stopword()

    def read_in_stopword(self):
        file_obj = codecs.open(self.stopword_filepath, 'r', 'utf-8')
        while True:
            line = file_obj.readline()
            line=line.strip('\r\n')
            if not line:
                break
            self.stopwords.add(line)
        file_obj.close()

    #tokenize, remove stop words, and stemming using Porter Stemmer  
    def cut(self, sentence, stopword= False, stemming = True):
        seg_list = nltk.word_tokenize(sentence)
        results = []
        if stopword:
            for seg in seg_list:
                if seg in self.stopwords:
                    continue
                if seg.isalpha():
                    results.append(seg)
        else:
            results=[token for token in seg_list if token.isalpha()]
        if stemming:
            porter = nltk.PorterStemmer()
            results=[porter.stem(token.lower()) for token in results]
        return results

In [5]:
#generate class for sentences
class Sentence(object):
    def __init__(self, sentence, seg, id=0):
        self.id = id
        self.origin_sentence = sentence
        self.cuted_sentence = self.cut(seg)

    # sentence segmentation
    def cut(self, seg):
        return seg.cut(self.origin_sentence)

    # get words after sentence segmentation
    def get_cuted_sentence(self):
        return self.cuted_sentence

    def get_origin_sentence(self):
        return self.origin_sentence

    # set scores for sentences
    def set_score(self, score):
        self.score = score

In [6]:
#generate class for calculating similarity
class SentenceSimilarity():
    def __init__(self, seg, csName):
        self.seg = seg
        self.csName = csName.lower()

    def set_sentences(self, sentences):
        self.sentences = []
        for i in range(0, len(sentences)):
            self.sentences.append(Sentence(sentences[i], self.seg, i))

    # get words after sentence segmentation
    def get_cuted_sentences(self):
        cuted_sentences = []
        for sentence in self.sentences:
            cuted_sentences.append(sentence.get_cuted_sentence())
        return cuted_sentences

    # using basic model to build complicated models
    def simple_model(self, min_frequency = 1):
        self.texts = self.get_cuted_sentences()

        # remove words with lowest frequency
        frequency = defaultdict(int)
        for text in self.texts:
            for token in text:
                frequency[token] += 1        
        self.texts = [[token for token in text if (frequency[token] > min_frequency) and (token != self.csName)] for text in self.texts]
        # generate dictionary class
        self.dictionary = corpora.Dictionary(self.texts)
        # build a corpus
        self.corpus_simple = [self.dictionary.doc2bow(text) for text in self.texts]
    
    def average_word_vectors(self,words, model, vocabulary, num_features):    
        feature_vector = np.zeros((num_features,),dtype="float64")
        nwords = 0.
        for word in words:
            if word in vocabulary: 
                nwords = nwords + 1.
                feature_vector = np.add(feature_vector, model[word])
        if nwords:
            feature_vector = np.divide(feature_vector, nwords)
        return feature_vector 
   
    # averaged word vector features 
    def averaged_word_vectorizer(self,corpus, model, num_features):
        vocabulary = set(model.wv.index2word)
        if type(corpus[0])==list:
            features = [self.average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                            for tokenized_sentence in corpus]
        else:
            features=self.average_word_vectors(corpus, model, vocabulary, num_features)
        return np.array(features)
        
    # build word2vec model      
    def w2vModel(self):
        self.simple_model()
        # switch from simple model to comprehensive
        self.model = models.Word2Vec(self.texts,size=200, min_count=5)
        self.features = self.averaged_word_vectorizer(corpus=self.texts,
                                                 model=self.model,
                                                 num_features=200)           
    # build tfidf model
    def TfidfModel(self):
        self.simple_model()
        # switch from simple model to comprehensive
        self.model = models.TfidfModel(self.corpus_simple)
        self.corpus = self.model[self.corpus_simple]
        # Generate Similarity Matrix for TFIDF model
        self.index = similarities.MatrixSimilarity(self.corpus)

    # lsi model
    def LsiModel(self):
        self.simple_model()
        # switch from simple model to comprehensive
        self.model = models.LsiModel(self.corpus_simple)
        self.corpus = self.model[self.corpus_simple]
        # Generate Similarity Matrix for LSI model
        self.index = similarities.MatrixSimilarity(self.corpus)

    # lda model
    def LdaModel(self):
        self.simple_model()
        # switch from simple model to comprehensive
        self.model = models.LdaModel(self.corpus_simple)
        self.corpus = self.model[self.corpus_simple]
        # Generate Similarity Matrix for LDA model
        self.index = similarities.MatrixSimilarity(self.corpus)

    # preliminary steps for input sentences
    def sentence2vec(self, sentence):
        sentence = Sentence(sentence, self.seg).get_cuted_sentence()
        vec_bow = self.dictionary.doc2bow(sentence)
        return self.model[vec_bow]
    
    def bow2vec(self):
        vec = []
        length = max(self.dictionary) + 1
        for content in self.corpus:
            sentence_vectors = np.zeros(length)
            for co in content:
                sentence_vectors[co[0]] = co[1]  # 将句子出现的单词的tf-idf表示放入矩阵中
            vec.append(sentence_vectors)
        return vec

    # look for the most similar sentences
    # input: test sentence    
    def cosine_similarity(self,x,y):
        num = x.dot(y.T)
        denom = np.linalg.norm(x) * np.linalg.norm(y)
        return num / denom
    
    def similarity_k(self, sentence, k):        
        sentence_vec = self.sentence2vec(sentence)
        sims = self.index[sentence_vec]
        sim_k = sorted(enumerate(sims), key=lambda item: item[1], reverse=True)[:k]
        indexs = [i[0] for i in sim_k]
        scores = [i[1] for i in sim_k]
        return indexs, scores
    
    def similarity_v(self, sentence, k):       
        cuts=Sentence(sentence, self.seg).get_cuted_sentence()
        sentence_vec=self.averaged_word_vectorizer(corpus=cuts,
                                      model=self.model,
                                     num_features=200)
        d=[]
        for i in range(len(self.features)):
            score=self.cosine_similarity(self.features[i],sentence_vec)
            if score >=0 or score <=0:
                d.append([i,score]) 
        sim_k = sorted(d, key=lambda item: item[1], reverse=True)[:k]
        indexs = [i[0] for i in sim_k]
        scores = [i[1] for i in sim_k]
        return indexs, scores

In [7]:
def read_corpus(df,seg):
    qList = []
    # list of keywords in tweets
    qList_kw = []
    aList = []
    data = df[['text_y','text_x']]
    data_ls = np.array(data).tolist()
    for t in data_ls:
        qList.append(t[0])
        qList_kw.append(seg.cut(t[0]))
        aList.append(t[1])
    return qList_kw, qList, aList

# define function for frequency distribution plot
def plot_words(wordList):
    fDist = FreqDist(wordList)
    #print(fDist.most_common())
    print("Total number of words: ",fDist.N())
    print("Total number of distinct words: ",fDist.B())
    fDist.plot(10)

# Get the data and take a look

In [8]:
# Read data
df=pd.read_csv('C:/Users/yongl/Fordham/Spring 2020/Text Analytics/Text project/customer-support-on-twitter/twcs/twcs.csv')

In [9]:
#preview data
df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


In [10]:
#sort by companies responded the most
df[df.inbound==False].groupby('author_id').count().sort_values('text',ascending=False)[:10]

,tweet_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
author_id,,,,,,
AmazonHelp,169840,169840,169840,169840,85274,169287
AppleSupport,106860,106860,106860,106860,31564,106719
Uber_Support,56270,56270,56270,56270,18036,56261
SpotifyCares,43265,43265,43265,43265,13786,43243
Delta,42253,42253,42253,42253,12014,42197
Tesco,38573,38573,38573,38573,11148,38501
AmericanAir,36764,36764,36764,36764,14556,36598
TMobileHelp,34317,34317,34317,34317,9759,34287
comcastcares,33031,33031,33031,33031,7625,33007


In [11]:
#selected the three most responsive companies
clist=df[df.inbound==False].groupby('author_id').count().sort_values('text',ascending=False)[:3].index.to_list()

In [12]:
clist

['AmazonHelp', 'AppleSupport', 'Uber_Support']

In [13]:
# join request and response into the same row: 'text_x' is the response from customer services; 'text_y' is the request
def pre(df,author):
    a=df[df.author_id==author]
    a=a.merge(df.loc[:,['tweet_id','text']],left_on='in_response_to_tweet_id',right_on='tweet_id')
    a=a[a.response_tweet_id.isnull()]    
    a['text_x']=a.text_x.apply(lambda x: x[:x.find('@')]+x[x.find('@')+8:])    
    return a

In [14]:
# a list of tweets responded by 'AmazonHelp', 'AppleSupport', and 'Uber_Support'
dataset=[]
for i in clist:
    dataset.append(pre(df,i))
dataset

[        tweet_id_x   author_id  inbound                      created_at  \
 2              275  AmazonHelp    False  Wed Nov 22 10:06:26 +0000 2017   
 3              324  AmazonHelp    False  Wed Nov 22 09:06:00 +0000 2017   
 6              620  AmazonHelp    False  Tue Oct 31 22:28:34 +0000 2017   
 8              625  AmazonHelp    False  Tue Oct 31 22:34:32 +0000 2017   
 10             629  AmazonHelp    False  Wed Nov 01 12:53:34 +0000 2017   
 ...            ...         ...      ...                             ...   
 168818     2987819  AmazonHelp    False  Wed Nov 22 06:19:27 +0000 2017   
 168819     2987821  AmazonHelp    False  Wed Nov 22 06:17:26 +0000 2017   
 168820     2987900  AmazonHelp    False  Wed Nov 22 07:09:54 +0000 2017   
 168821     2987902  AmazonHelp    False  Wed Nov 22 07:09:53 +0000 2017   
 168822     2987938  AmazonHelp    False  Wed Nov 22 07:49:52 +0000 2017   
 
                                                    text_x response_tweet_id  \
 2    

# Run the Chatbot

In [ ]:
while True:
    way=input('Select a model for encoding: w2v, tfidf, lsi, or lda? ')
    #Set up for specific model
    w2v=False
    if way == 'w2v':
        w2v=True
        print('Please wait for the system to set up.')
        time1=time.time()
        seg0 = Seg()
        List_kw0, questionList0, answerList0 = read_corpus(dataset[0],seg0)
        ss0 = SentenceSimilarity(seg0,clist[0])
        ss0.set_sentences(questionList0)
        ss0.w2vModel()
#         ss0.TfidfModel() 
        #     ss.LsiModel()
        #     ss.LdaModel()        
        seg1 = Seg()
        List_kw1, questionList1, answerList1 = read_corpus(dataset[1],seg1)
        ss1 = SentenceSimilarity(seg1,clist[1])
        ss1.set_sentences(questionList1)
        ss1.w2vModel()
#         ss1.TfidfModel() 
        seg2 = Seg()
        List_kw2, questionList2, answerList2 = read_corpus(dataset[2],seg2)
        ss2 = SentenceSimilarity(seg2,clist[2])
        ss2.set_sentences(questionList2)
        ss2.w2vModel()
#         ss2.TfidfModel() 
        time2=time.time()
        print('The setup is now complete, which took {} s.'.format(time2-time1))
        print('---------------------------------------------------------------------------------------------------')
        
        #Start chatting with customer
        print('Hey there! This is Auto Customer Service. First, please choose the company you would like to chat with:')
        while True:
            company=input("0 for Amazon, 1 for Apple, 2 for Uber, 'q' to quit: ")
            if company in ['0','1','2']:
                while True:
                    question = input("Please type your question here or press 'q' to quit: ")
                    if question == 'q':
                        break
                    time1 = time.time()

                    # chats for Amazon
                    if company=='0':
                        question_k = ss0.similarity_v(question, 5)
                        time2 = time.time()
                        for i in range(5):
                            if question_k[1][i]>=0.5:
                                print('Thanks for asking, here is the',i+1,'most likely answer(s) from AmazonHelp:')
                                print(answerList0[question_k[0][i]])
                                while True:
                                    solved=input('Is your problem solved? (input y/n)')
                                    if solved=='n' or solved=='y':
                                        print()
                                        break
                                    elif solved!='y':
                                        input('Please enter y/n')
                                        continue
                                if solved=='y':
                                    print('Thank you for using Auto Customer Service. It\'s my pleasure to help.')
                                    break

                            else:
                                if i == 0:
                                    print('Unfortunately, I can\'t find any answer in our database system, please contact human services.' )
                                else:
                                    print('Unfortunately, I can\'t find more answers in the system, please contact human services if you still have questions.')
                                break


                        for idx, score in zip(*question_k):
                            print("Similar questions： {},                score： {}".format(questionList0[idx], score))

                        cost = time2 - time1
                        print('It took {} s to look for the answers'.format(cost))
                        print('---------------------------------------------------------------------------------------------------')
                        print('Is there anything else I can help?')# chats for apple
                    elif company=='1':
                        question_k = ss1.similarity_v(question, 5)
                        time2 = time.time()
                        for i in range(5):
                            if question_k[1][i]>=0.5:
                                print('Thanks for asking, here is the',i+1,'most likely answer from AppleSupport')
                                print(answerList1[question_k[0][i]])
                                while True:
                                    solved=input('Is this problem solved? (input y/n)')
                                    if solved=='n' or solved=='y':
                                        print()
                                        break
                                    elif solved!='y':
                                        input('Please enter y/n')
                                        continue
                                if solved=='y':
                                    print('Thank you for using Auto Customer Service. It\'s my pleasure to solve your problem.')
                                    break

                            else:
                                if i == 0:
                                    print('Unfortunately, I can\'t find any answer in our database, please contact human services.' )
                                else:
                                    print('Unfortunately, I can\'t find more answers in our database, please contact human services if you still have questions.')
                                break


                        for idx, score in zip(*question_k):
                            print("Similar questions： {},                score： {}".format(questionList1[idx], score))

                        cost = time2 - time1
                        print('It took {} s to look for the answers'.format(cost))
                        print('---------------------------------------------------------------------------------------------------')
                        print('Is there anything else I can help?')

                    # chats for Uber
                    else:
                        question_k = ss2.similarity_v(question, 5)
                        time2 = time.time()
                        for i in range(5):
                            if question_k[1][i]>=0.5:
                                print('Thanks for asking, here is the',i+1,'most likely answer(s) from Uber_Support')
                                print(answerList2[question_k[0][i]])
                                while True:
                                    solved=input('Is this problem solved? (input y/n)')
                                    if solved=='n' or solved=='y':
                                        print()
                                        break
                                    elif solved!='y':
                                        input('Please enter y/n')
                                        continue
                                if solved=='y':
                                    print('Thank you for using Auto Customer Service. It\'s my pleasure to solve your problem.')
                                    break

                            else:
                                if i == 0:
                                    print('Unfortunately, I can\'t find any answer in our database, please contact human services.' )
                                else:
                                    print('Unfortunately, I can\'t find more answers in our database, please contact human services if you still have questions.')
                                break

                        for idx, score in zip(*question_k):
                            print("Similar questions： {},                score： {}".format(questionList2[idx], score))

                        cost = time2 - time1
                        print('It took {} s to look for the answers'.format(cost))
                        print('---------------------------------------------------------------------------------------------------')
                        print('Is there anything else I can help?')
                print('Thank you for asking. Would you like to ask questions about other companies?')
                
                    
            elif company=='q':
                print('Thank you. Say safe and have a good one!')
                break
            else:
                print('Please input 0, 1, 2 or q')
            
        break
            
            
        
    elif way == 'tfidf':
        print('Please wait for system to set up')
        time1=time.time()
        seg0 = Seg()
        List_kw0, questionList0, answerList0 = read_corpus(dataset[0],seg0)
        ss0 = SentenceSimilarity(seg0,clist[0])
        ss0.set_sentences(questionList0)
        #ss0.w2vModel()
        ss0.TfidfModel() 
        #     ss.LsiModel()
        #     ss.LdaModel()        
        seg1 = Seg()
        List_kw1, questionList1, answerList1 = read_corpus(dataset[1],seg1)
        ss1 = SentenceSimilarity(seg1,clist[1])
        ss1.set_sentences(questionList1)
#         ss1.w2vModel()
        ss1.TfidfModel() 
        seg2 = Seg()
        List_kw2, questionList2, answerList2 = read_corpus(dataset[2],seg2)
        ss2 = SentenceSimilarity(seg2,clist[2])
        ss2.set_sentences(questionList2)
#         ss2.w2vModel
        ss2.TfidfModel() 
        time2=time.time()
        print('Finished! Time cost for setting up: {} s'.format(time2-time1))
        print('---------------------------------------------------------------------------------------------------')
        break
        
    elif way == 'lsi':
        print('Please wait for system to set up')
        time1=time.time()
        seg0 = Seg()
        List_kw0, questionList0, answerList0 = read_corpus(dataset[0],seg0)
        ss0 = SentenceSimilarity(seg0,clist[0])
        ss0.set_sentences(questionList0)
        #ss0.w2vModel()
#         ss0.TfidfModel() 
        ss0.LsiModel()
        #     ss.LdaModel()        
        seg1 = Seg()
        List_kw1, questionList1, answerList1 = read_corpus(dataset[1],seg1)
        ss1 = SentenceSimilarity(seg1,clist[1])
        ss1.set_sentences(questionList1)
#         ss1.w2vModel()
        ss1.LsiModel() 
        seg2 = Seg()
        List_kw2, questionList2, answerList2 = read_corpus(dataset[2],seg2)
        ss2 = SentenceSimilarity(seg2,clist[2])
        ss2.set_sentences(questionList2)
#         ss2.w2vModel
        ss2.LsiModel() 
        time2=time.time()
        print('Finished! Time cost for setting up: {} s'.format(time2-time1))
        print('---------------------------------------------------------------------------------------------------')
        break
        
    elif way == 'lda':
        print('Please wait for system to set up')
        time1=time.time()
        seg0 = Seg()
        List_kw0, questionList0, answerList0 = read_corpus(dataset[0],seg0)
        ss0 = SentenceSimilarity(seg0,clist[0])
        ss0.set_sentences(questionList0)
        #ss0.w2vModel()
#         ss0.TfidfModel() 
        #     ss.LsiModel()
        ss0.LdaModel()        
        seg1 = Seg()
        List_kw1, questionList1, answerList1 = read_corpus(dataset[1],seg1)
        ss1 = SentenceSimilarity(seg1,clist[1])
        ss1.set_sentences(questionList1)
#         ss1.w2vModel()
        ss1.LdaModel() 
        seg2 = Seg()
        List_kw2, questionList2, answerList2 = read_corpus(dataset[2],seg2)
        ss2 = SentenceSimilarity(seg2,clist[2])
        ss2.set_sentences(questionList2)
#         ss2.w2vModel
        ss2.LdaModel() 
        time2=time.time()
        print('Finished! Time cost for setting up: {} s'.format(time2-time1))
        print('---------------------------------------------------------------------------------------------------')
        break
    else:
        print('Please input right')
        continue
        
if w2v==False:
    print('Hey there! This is Auto Customer Service. First please choose the company:')
    while True:
        company=input('0 for Amazon, 1 for Apple, 2 for Uber: ')
        if company in ['0','1','2']:
            while True:        
                question = input("Please type your question here ('q' to quit): ")
                if question == 'q':
                    break
                time1 = time.time()
                if company=='0':           
                    question_k = ss0.similarity_k(question, 5)
                    time2 = time.time()
                    for i in range(5):
                        if question_k[1][i]>=0.5:
                            print('Thanks for asking, here is the',i+1,'most likely answer(s) from AmazonHelp')
                            print(answerList0[question_k[0][i]])
                            while True:
                                solved=input('Is this problem solved? (input y/n)')
                                if solved=='n' or solved=='y':
                                    print()
                                    break
                                elif solved!='y':
                                    input('Please enter y/n')
                                    continue
                            if solved=='y':
                                print('Thank you for using Auto Customer Service. It\'s my pleasure to solve your problem.')
                                break                    
                        else:
                            if i == 0:
                                print('Unfortunately, I can\'t find any answer in our database, please contact human services.' )
                            else:
                                print('Unfortunately, I can\'t find more answers in our database, please contact human services if you still have questions.')
                            break


                    for idx, score in zip(*question_k):
                        print("Similar questions： {},                score： {}".format(questionList0[idx], score))

                    cost = time2 - time1
                    print('It took {} s to look for the answers'.format(cost))
                    print('---------------------------------------------------------------------------------------------------')
                    print('What else can I help')
                elif company=='1':
                    question_k = ss1.similarity_k(question, 5)
                    time2 = time.time()
                    for i in range(5):
                        if question_k[1][i]>=0.5:
                            print('Thanks for asking, here is the',i+1,'most likely answer(s) from AppleSupport')
                            print(answerList1[question_k[0][i]])
                            while True:
                                solved=input('Is this problem solved? (input y/n)')
                                if solved=='n' or solved=='y':
                                    print()
                                    break
                                elif solved!='y':
                                    input('Please enter y/n')
                                    continue
                            if solved=='y':
                                print('Thank you for using Auto Customer Service. It\'s my pleasure to solve your problem.')
                                break

                        else:
                            if i == 0:
                                print('Unfortunately, I can\'t find any answer in our database, please contact human services.' )
                            else:
                                print('Unfortunately, I can\'t find more answers in our database, please contact human services if you still have questions.')
                            break


                    for idx, score in zip(*question_k):
                        print("Similar questions： {},                score： {}".format(questionList1[idx], score))

                    cost = time2 - time1
                    print('It took {} s to look for the answers'.format(cost))
                    print('---------------------------------------------------------------------------------------------------')
                    print('Is there anything else I can help?')
                else:
                    question_k = ss2.similarity_k(question, 5)
                    time2 = time.time()
                    for i in range(5):
                        if question_k[1][i]>=0.5:
                            print('Thanks for asking, here is the',i+1,'most likely answer(s) from Uber_Support')
                            print(answerList2[question_k[0][i]])
                            while True:
                                solved=input('Is this problem solved? (input y/n)')
                                if solved=='n' or solved=='y':
                                    print()
                                    break
                                elif solved!='y':
                                    input('Please enter y/n')
                                    continue
                            if solved=='y':
                                print('Thank you for using Auto Customer Service. It\'s my pleasure to solve your problem.')
                                break

                        else:
                            if i == 0:
                                print('Unfortunately, I can\'t find any answer in the systen, please contact human services.' )
                            else:
                                print('Unfortunately, I can\'t find more answers in the systen, please contact human services if you still have questions.')
                            break


                    for idx, score in zip(*question_k):
                        print("Similar questions： {},                score： {}".format(questionList2[idx], score))

                    cost = time2 - time1
                    print('It took {} s to look for the answers?'.format(cost))
                    print('---------------------------------------------------------------------------------------------------')
                    print('Is there anything else I can help')
            
            print('Thank you for asking. Do you want to ask questions about other companies?')
                
                    
        elif company=='q':
            print('Thank you. Stay safe and have a good one!')
            break
        else:
            print('Please enter 0, 1, 2 or q')

Select a model for encoding: w2v, tfidf, lsi, or lda? LDA
Please input right
Select a model for encoding: w2v, tfidf, lsi, or lda? lda
Please wait for system to set up
Finished! Time cost for setting up: 671.035742521286 s
---------------------------------------------------------------------------------------------------
Hey there! This is Auto Customer Service. First please choose the company:
0 for Amazon, 1 for Apple, 2 for Uber: 0
Please type your question here ('q' to quit): I selected free two day delivery but the package arrived late
Thanks for asking, here is the 1 most likely answer(s) from AmazonHelp
Hi, you can leave your comments and submit additional feedback about packaging here: https://t.co/fxgZ6hTrFA.^MA
Is this problem solved? (input y/n)n

Thanks for asking, here is the 2 most likely answer(s) from AmazonHelp
I'm so sorry for the delay. If you have a moment, please contact us by phone or chat here: : https://t.co/hApLpMlfHN. ^LH
Is this problem solved? (input y/n)n



In [ ]:
I selected free two day delivery but the package arrived late

# Topic Modeling using LDA

In [58]:
amazon = []
amazon.append(pre(df,'AmazonHelp'))
amazon

[        tweet_id_x   author_id  inbound                      created_at  \
 2              275  AmazonHelp    False  Wed Nov 22 10:06:26 +0000 2017   
 3              324  AmazonHelp    False  Wed Nov 22 09:06:00 +0000 2017   
 6              620  AmazonHelp    False  Tue Oct 31 22:28:34 +0000 2017   
 8              625  AmazonHelp    False  Tue Oct 31 22:34:32 +0000 2017   
 10             629  AmazonHelp    False  Wed Nov 01 12:53:34 +0000 2017   
 ...            ...         ...      ...                             ...   
 168818     2987819  AmazonHelp    False  Wed Nov 22 06:19:27 +0000 2017   
 168819     2987821  AmazonHelp    False  Wed Nov 22 06:17:26 +0000 2017   
 168820     2987900  AmazonHelp    False  Wed Nov 22 07:09:54 +0000 2017   
 168821     2987902  AmazonHelp    False  Wed Nov 22 07:09:53 +0000 2017   
 168822     2987938  AmazonHelp    False  Wed Nov 22 07:49:52 +0000 2017   
 
                                                    text_x response_tweet_id  \
 2    

In [ ]:
q = amazon[-1]['text_y'].to_list()
q

In [ ]:
#remove '\r' and punctuations 
new_text = []
for tweet in q: 
    for i in tweet:
        if i in string.punctuation or i == '@':
            tweet = tweet.replace(i,'') #replace punctuation with nothing
        if i == '\r':
            tweet = tweet.replace(i,' ') #replace \r with space
    new_text.append(tweet) 
new_text

In [ ]:
#remove stop words, to lowercase and tokenize
mystopwords = stopwords.words()
tokens_list = [[word for word in tweet.lower().split(' ') if word not in mystopwords and word.isalpha() and word != 'amazon' and word != 'amazonhelp']
         for tweet in new_text]
tokens_list

In [ ]:
#remove words that appear only once
frequency = defaultdict(int)

for tokens in tokens_list:
    for token in tokens:
        frequency[token] += 1      
tokens_list = [[token for token in tokens if frequency[token]>1]
              for tokens in tokens_list]
tokens_list

In [ ]:
#Generate Term Document Matrix
# generate token dictionary class
dictionary = corpora.Dictionary(tokens_list) 
print(dictionary)

In [ ]:
# generate a unique token list 
sort_token = sorted(dictionary.items(),key=lambda k:k[0], reverse = False)
unique_token = [token for (ID,token) in sort_token]

# build a corpus
corpus = [dictionary.doc2bow(tokens) for tokens in tokens_list]
corpus

In [ ]:
# Save a Term Document Matrix
matrix = gensim.matutils.corpus2dense(corpus,num_terms=len(dictionary),dtype = 'int')
matrix = matrix.T 
#transpose the matrix 

#convert the numpy matrix into pandas dataframe
matrix_df = pd.DataFrame(matrix, columns=unique_token)
matrix_df

In [ ]:
#fit to LDA model
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=10) 

#Topic matrix (V matrix)
lda.print_topics(10) 

In [ ]:
# Generate U Matrix for LDA model
corpus_lda = lda[corpus] 
#transform lda model

#convert corpus_lda to numpy matrix
U_matrix_lda = gensim.matutils.corpus2dense(corpus_lda,num_terms=10).T

#write U_matrix into pandas dataframe and output
U_matrix_lda_df = pd.DataFrame(U_matrix_lda)
U_matrix_lda_df

In [ ]:
print (matrix_df.shape)
print (U_matrix_lda_df.shape)

In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, dictionary)